In [60]:
import glob
import os.path
import xarray as xr
import pandas as pd
import argparse
import cftime


#def time_coord(ds):
#    '''Create a proper time coordinate for the ocean output from Mk3L.
#    ds: xarray dataset from reading Mk3L ocean output(s)'''

#    year = list(range(ds.sizes['month']//12))
#    year = [str(i+1800)+'{:0>2}'.format(m+1) for i in year for m in range(12)]
#    time = 
#    time=pd.to_datetime(year, format="%Y%m")

#    return time

def concat_ocean(path, file_template):
    fname = os.path.join(path, file_template)
    file_list = sorted(glob.glob(fname))

    ds = xr.open_mfdataset(file_list, concat_dim='month',decode_cf=False)
    # New time coordinate to include the year. 
    # Arbitrary add 1800 to make it easier to deal with the date! (Calendar issues)
    ds.load()
    time=xr.cftime_range(start='0001', periods=600, freq='MS', calendar='noleap')
    
    # Replace the month coordinate with these new values and rename to time
    ds = ds.where(ds > -1.e34)
    ds = ds.assign_coords(month=time).rename(month='time')
    return ds



In [ ]:
#if __name__ == '__main__':
#    # User inputs:
#    parser = argparse.ArgumentParser()
#    parser.add_argument('-p',default='./',help='Path with the Mk3L ocean outputs', dest='path')
#    parser.add_argument('-r',default='ctl01', help='Name of the experiment', dest='run')
#
#    args = parser.parse_args()
file_template = 'com.'+'ctl01'+'.[0-9]*.nc'
print(f"Reading all files: {file_template}")

test = concat_ocean('/Users/Claire/mount/raij_home/gdata5/WS2019/ccc561/ctl01/', file_template)
print("Data loaded")
clim = test.groupby('time.year').mean(dim='time')  # Annual mean
dd = {var: var+"_clim" for var in clim.data_vars}  # Rename clim variables
clim = clim.rename(dd)
print("Mean calculated")
for var in test.data_vars:
    fname = os.path.join('/Users/Claire/mount/raij_home/gdata5/WS2019/ccc561/ctl01/',f'com.ctl01.{var}_all_yr.nc')
    print(f"Writing {var}")
    ds = xr.merge([test[var], clim[var+"_clim"]])
    ds.to_netcdf(fname)


In [57]:
test['smfzon'][0,0:10,0:10].values

array([[-0.07291944, -0.0735697 , -0.08073891, -0.13023704, -0.15982366,
        -0.16995318, -0.16666539, -0.15906148, -0.15088224, -0.14623605],
       [-0.0735697 , -0.0735697 , -0.09759142, -0.13876007, -0.15982366,
        -0.16995318, -0.16666538, -0.15906148, -0.15088221, -0.14623605],
       [-0.07356971, -0.07674401, -0.11749247, -0.13876007, -0.15982366,
        -0.1699532 , -0.16666539, -0.15906149, -0.15088223, -0.14623608],
       [-0.0735697 , -0.09353814, -0.12309051, -0.13876007, -0.15982366,
        -0.1699532 , -0.16666539, -0.15906148, -0.15088223, -0.14623606],
       [-0.07899316, -0.10980853, -0.12309051, -0.13876007, -0.15982366,
        -0.16995318, -0.16666538, -0.15906148, -0.15088221, -0.14623605],
       [-0.09390692, -0.11086471, -0.12309051, -0.13876007, -0.15982366,
        -0.16995318, -0.16666539, -0.15906148, -0.15088223, -0.14623606],
       [-0.09948184, -0.11086471, -0.12309051, -0.13876007, -0.15982366,
        -0.1699532 , -0.16666538, -0.15906148

{'itt': 'itt_clim',
 'dtts': 'dtts_clim',
 'relyr': 'relyr_clim',
 'kmt': 'kmt_clim',
 'kmu': 'kmu_clim',
 'smfzon': 'smfzon_clim',
 'smfmer': 'smfmer_clim',
 'stfht': 'stfht_clim',
 'stfsal': 'stfsal_clim',
 'temp': 'temp_clim',
 'sal': 'sal_clim',
 'rho': 'rho_clim',
 'u': 'u_clim',
 'v': 'v_clim',
 'w': 'w_clim',
 'uedd': 'uedd_clim',
 'vedd': 'vedd_clim',
 'wedd': 'wedd_clim',
 'res': 'res_clim',
 'cdepthm': 'cdepthm_clim',
 'mola': 'mola_clim',
 'molp': 'molp_clim',
 'moli': 'moli_clim',
 'molg': 'molg_clim',
 'moea': 'moea_clim',
 'moep': 'moep_clim',
 'moei': 'moei_clim',
 'moeg': 'moeg_clim',
 'mota': 'mota_clim',
 'motp': 'motp_clim',
 'moti': 'moti_clim',
 'motg': 'motg_clim'}

<xarray.Dataset>
Dimensions:       (latts: 112, latuv: 112, lonts: 128, lonuv: 128, year: 50, zts: 21, zuv: 21)
Coordinates:
  * lonts         (lonts) float32 -1.40625 1.40625 ... 352.96875 355.78125
  * lonuv         (lonuv) float32 0.0 2.8125 5.625 ... 351.5625 354.375 357.1875
  * latts         (latts) float32 -88.34722 -86.775444 ... 86.775444 88.34722
  * latuv         (latuv) float32 -87.56133 -85.98956 ... 87.56133 89.13311
  * zts           (zts) float32 12.5 37.5 65.0 98.5 ... 3475.0 3925.0 4375.0
  * zuv           (zuv) float32 0.0 25.0 50.0 80.0 ... 3250.0 3700.0 4150.0
  * year          (year) int64 1 2 3 4 5 6 7 8 9 ... 42 43 44 45 46 47 48 49 50
Data variables:
    itt_clim      (year) float64 dask.array<shape=(50,), chunksize=(1,)>
    dtts_clim     (year) timedelta64[ns] 01:00:00 01:00:00 ... 01:00:00 01:00:00
    relyr_clim    (year) float32 dask.array<shape=(50,), chunksize=(1,)>
    kmt_clim      (year, latts, lonts) float64 dask.array<shape=(50, 112, 128), chunksize